In [1]:
from typing import Dict
from core_components.entities.library import AIEntity, AICharactor, MobCharactor
from core_components.events.base import BaseGameEvent
from core_components.events.library import AIEvent
from state import GameState
from core_components.handlers.library import MobHandler

b = MobCharactor(name="Test", color=(255, 255, 255), symbol='@')
a = MobHandler(entity=b)
a.state = GameState()
b._ai = a

c:\Users\jason\workspaces\repos\jrl\src\core_components\tiles\base.py:469: UserWarning: Cannot align corners without 'center', 'width', and 'height' attributes set.
  warn("Cannot align corners without 'center', 'width', and 'height' attributes set.", UserWarning)


In [2]:
b.is_spotting = True
b.is_targeting = True
b.target = b
b.ai.update_state()
b.ai.state.events.qsize()


1

In [ ]:
ev = a.get_event_from_state_vector(v)
ev

In [ ]:
from typing import Protocol, runtime_checkable
@runtime_checkable
class C(Protocol):
    message: str

class D(C):
    message: str

    def __init__(self, message: str = "D Event Triggered") -> None:
        self.message = message

d = D()

isinstance(d, D)


In [3]:
g = AIEvent(None, None, None)

In [5]:
isinstance(g, AIEvent)

True

In [ ]:
import numpy as np
c = np.array((  (0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1),
                                                        (0, 1, 0, 0), (0, 1, 0, 1), (0, 1, 1, 0), (0, 1, 1, 1),
                                                        (1, 0, 0, 0), (1, 0, 0, 1), (1, 0, 1, 0), (1, 0, 1, 1),
                                                        (1, 1, 0, 0), (1, 1, 0, 1), (1, 1, 1, 0), (1, 1, 1, 1) ))

In [ ]:
d = (  (None), (None), (None), (None),
                                                    (None), (None), (None), (None),
                                                    (None), (None), ('AcquireTarget'), ('EngageTarget'),
                                                    (None), (None), ('AcquireTarget'), ('EngageTarget') )

In [ ]:
vector = (1, 0, 1, 0)
index = np.where((c == vector).all(axis=1))[0]
event_name = d[index[0]]
d[index[0]]

In [ ]:
c = AIEvent()


In [ ]:
c.__class__

In [ ]:
import tcod.context
import tcod.event

# ... tileset loading code here ...

def main() -> None:
    # Create the context
    with tcod.context.new_window(width=800, height=600, title="My TCOD Context") as context:
        while True:
            # Create a new console every frame or when resized to match context size
            # Its width and height will be based on the window's current size and tile size
            console = context.new_console()

            # Access the console dimensions (in tiles)
            con_width = console.width
            con_height = console.height
            console.print(x=0, y=0, string=f"Console size: {con_width}x{con_height}")

            # Access the window dimensions (in pixels)
            win_width, win_height = context.sdl_window.size
            console.print(x=0, y=1, string=f"Window size (pixels): {win_width}x{win_height}")

            # Present the console
            context.present(console)

            # Handle events
            for event in tcod.event.wait():
                if isinstance(event, tcod.event.Quit):
                    raise SystemExit
                # You can also detect window resize events directly
                # case tcod.event.WindowResized(width=width, height=height):
                #     pass # The next call to context.new_console will have the new size

if __name__ == "__main__":
    main()
